In [1]:
import pandas as pd
from datetime import datetime
from os.path import join
import json 

In [5]:

def load_json_data(json_file):
    path = join('data', 'json', json_file)
    with open(path, "r", encoding="utf-8") as file:
        datos = json.load(file)
    return datos

def load_txt_data(txt_file):
    path = join('data', 'txt', txt_file)
    with open(path, "r", encoding="utf-8") as file:
        datos = [line.strip('\n').split(': ') for line in file.readlines()]
        datos = {line[0]: int(line[1]) for line in datos}
    return datos

In [6]:
region_columns = {
    'id_region' : [0,1],
    'nombre' : [1,2]
}
df_region = pd.DataFrame(region_columns)

In [7]:
frutas_columns = {
    'id_fruta' : [i for i in range(0, len(load_txt_data('precios.txt')))],
    'nombre' : [nombre for nombre in load_txt_data('precios.txt').keys()],
    'precio' : [precio for precio in load_txt_data('precios.txt').values()]
}
df_frutas = pd.DataFrame(frutas_columns)
df_frutas

,id_fruta,nombre,precio
0,0,kiwi,200
1,1,manzana,150
2,2,naranja,250
3,3,limón,150
4,4,melón,300


In [8]:
usuarios_ordenes_r1 = load_json_data('usuarios_ordenes_r1.json')
usuarios_ordenes_r2 = load_json_data('usuarios_ordenes_r2.json')

usuario_columns = {
    'id_usuario' : [],
}

usuario_orden_columns = {
    'id_usuario' : [],
    'id_orden' : [],
    'id_region' : [],
    'fecha' : [],
}

for id_region, usuario_ordenes_rX in enumerate([usuarios_ordenes_r1, usuarios_ordenes_r2]):
    for row in usuario_ordenes_rX['values']:
        id_usuario = row['usuario_id']
        id_orden = row['compra_id']

        if id_usuario not in usuario_columns['id_usuario']:
            usuario_columns['id_usuario'].append(id_usuario)

        usuario_orden_columns['id_usuario'].append(id_usuario)
        usuario_orden_columns['id_orden'].append(id_orden)
        usuario_orden_columns['id_region'].append(id_region)
        usuario_orden_columns['fecha'].append(row['fecha'])
        
        
    
df_usuarios = pd.DataFrame(usuario_columns)
df_usuario_orden = pd.DataFrame(usuario_orden_columns) 

In [9]:
df_usuarios

,id_usuario
0,1
1,2
2,3
3,4
4,5
...,...
255,257
256,259
257,260
258,261


In [10]:
df_usuario_orden['fecha'] = pd.to_datetime(df_usuario_orden['fecha'])
df_usuario_orden

,id_usuario,id_orden,id_region,fecha
0,1,554,0,2023-01-07
1,1,14,0,2023-01-09
2,1,284,0,2023-01-12
3,1,419,0,2023-01-16
4,1,149,0,2023-01-31
...,...,...,...,...
34433,255,1985,1,2023-12-13
34434,255,2445,1,2023-12-15
34435,255,145,1,2023-12-20
34436,255,2675,1,2023-12-27


In [11]:
ordenes_region1 = load_json_data('ordenes_r1.json')
ordenes_region2 = load_json_data('ordenes_r2.json')
compras_columns = {
    'id_compra' : [],
    'id_orden' : [],
    'id_fruta' : [],
    'mes' : [],
    'id_region' : [],
    'cantidad' : []
}

months = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}


def cargar_compras(compras_columns): 
    id_compra = 0
    regiones = [ordenes_region1, ordenes_region2]
    for id_region, ordenes_region in enumerate(regiones):
        for month in ordenes_region.keys():
            for order in ordenes_region[month]:
                id_orden = order['orden_id']
                del order['orden_id']
                for fruta in order.items():
                    fruta = fruta[0]
                    id_fruta = df_frutas.query(f'nombre == "{fruta}"')['id_fruta'].values[0]
                    cantidad = order[fruta]
                    
                    compras_columns['id_compra'].append(id_compra)
                    compras_columns['id_orden'].append(id_orden)
                    compras_columns['id_fruta'].append(id_fruta)
                    compras_columns['mes'].append(months[month])
                    compras_columns['id_region'].append(id_region)
                    compras_columns['cantidad'].append(cantidad)
                    
                    id_compra += 1
    
    return compras_columns

compras_columns = cargar_compras(compras_columns)
df_compras = pd.DataFrame(compras_columns)
df_compras

,id_compra,id_orden,id_fruta,mes,id_region,cantidad
0,0,1,1,1,0,8
1,1,1,3,1,0,14
2,2,2,0,1,0,14
3,3,3,2,1,0,11
4,4,4,0,1,0,11
...,...,...,...,...,...,...
65942,65942,2688,2,12,1,10
65943,65943,2689,2,12,1,7
65944,65944,2690,2,12,1,15
65945,65945,2691,2,12,1,8


In [12]:
df_ordenes_compras = pd.merge(df_usuario_orden, df_compras, on=['id_orden', 'id_region'], how='inner')


In [13]:
df_ordenes_compras_v2 = df_ordenes_compras[df_ordenes_compras['mes'] == df_ordenes_compras['fecha'].dt.month]

In [14]:
# Da exacto. 
df_ordenes_compras_v2.query('id_usuario == 1 and id_region == 0 and id_orden == 554')

,id_usuario,id_orden,id_region,fecha,id_compra,id_fruta,mes,cantidad
0,1,554,0,2023-01-07,902,4,1,8


In [15]:
df_ordenes_compras_v3 = df_ordenes_compras_v2.drop(columns=['mes', 'id_compra'])

# Recopilacion



In [16]:
df_ordenes_compras_v3

,id_usuario,id_orden,id_region,fecha,id_fruta,cantidad
0,1,554,0,2023-01-07,4,8
16,154,554,0,2023-02-07,4,11
32,78,554,0,2023-03-06,4,12
48,50,554,0,2023-05-23,1,8
49,50,554,0,2023-05-23,2,8
...,...,...,...,...,...,...
696788,227,2797,1,2023-10-28,2,11
696789,228,2838,1,2023-10-20,2,12
696790,229,2791,1,2023-10-26,2,3
696791,231,2765,1,2023-10-23,2,5


In [17]:
df_usuarios_v2 = df_usuarios.sort_values(by='id_usuario')
df_usuarios_v2


,id_usuario
0,1
1,2
2,3
3,4
4,5
...,...
254,256
255,257
256,259
257,260


In [18]:
df_frutas

,id_fruta,nombre,precio
0,0,kiwi,200
1,1,manzana,150
2,2,naranja,250
3,3,limón,150
4,4,melón,300


In [19]:
df_region

,id_region,nombre
0,0,1
1,1,2


# Archivos csv

In [21]:
df_region.to_csv('data/csv/region.csv', index=False)
df_frutas.to_csv('data/csv/frutas.csv', index=False)
df_usuarios_v2.to_csv('data/csv/usuarios.csv', index=False)
df_ordenes_compras_v3.to_csv('data/csv/ordenes_compras.csv', index=False)